##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Automatically upgrade code to TensorFlow 2

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/upgrade">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
    Download notebook</a>
  </td>
</table>


TensorFlow 2.0 includes many API changes, such as reordering arguments, renaming symbols, and changing default values for parameters. Manually performing all of these modifications would be tedious and prone to error. To streamline the changes, and to make your transition to TF 2.0 as seamless as possible, the TensorFlow team has created the `tf_upgrade_v2` utility to help transition legacy code to the new API.

Note: `tf_upgrade_v2` is installed automatically for TensorFlow 1.13 and later (including all TF 2.0 builds).

Typical usage is like this:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">
tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

It will accelerate your upgrade process by converting existing TensorFlow 1.x Python scripts to TensorFlow 2.0.

The conversion script automates as much as possible, but there are still syntactical and stylistic changes that cannot be performed by the script.

## Compatibility modules

Certain API symbols can not be upgraded simply by using a string replacement. To ensure your code is still supported in TensorFlow 2.0, the upgrade script includes a `compat.v1` module. This module replaces TF 1.x symbols like `tf.foo` with the equivalent `tf.compat.v1.foo` reference. While the compatibility module is nice, we recommend that you manually proofread replacements and migrate them to new APIs in the `tf.*` namespace instead of `tf.compat.v1` namespace as quickly as possible.

Because of TensorFlow 2.x module deprecations (for example, `tf.flags` and `tf.contrib`), some changes can not be worked around by switching to `compat.v1`. Upgrading this code may require using an additional library (for example, [`absl.flags`](https://github.com/abseil/abseil-py)) or switching to a package in [tensorflow/addons](http://www.github.com/tensorflow/addons).


## Recommended upgrade process

The rest of this guide demonstrates how to use the upgrade script. While the upgrade script is easy to use, it is strongly recomended that you use the script as part of the following process: 

1. **Unit Test**: Ensure that the code you’re upgrading has a unit test suite with reasonable coverage. This is Python code, so the language won’t protect you from many classes of mistakes. Also ensure that any dependency you have has already been upgraded to be compatible with TensorFlow 2.0.

1. **Install TensorFlow 1.14**: Upgrade your TensorFlow to the latest TensorFlow 1.x version, at least 1.14. This includes the final TensorFlow 2.0 API in `tf.compat.v2`.

1. **Test With 1.14**: Ensure your unit tests pass at this point. You’ll be running them repeatedly as you upgrade so starting from green is important.

1. **Run the upgrade script**: Run `tf_upgrade_v2` on your entire source tree, tests included. This will upgrade your code to a format where it only uses symbols available in TensorFlow 2.0. Deprecated symbols will be accessed with `tf.compat.v1`. These will eventually require manual attention, but not immediately.

1. **Run the converted tests with TensorFlow 1.14**: Your code should still run fine in TensorFlow 1.14. Run your unit tests again. Any error in your tests here means there’s a bug in the upgrade script. [Please let us know](https://github.com/tensorflow/tensorflow/issues).

1. **Check the upgrade report for warnings and errors**: The script writes a report file that explains any conversions you should double-check, or any manual action you need to take. For example: Any remaining instances of contrib will require manual action to remove. Please consult [the RFC for more instructions](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md). 

1. **Install TensorFlow 2.0**: At this point it should be safe to switch to TensorFlow 2.0

1. **Test with `v1.disable_v2_behavior`**: Re-running your tests with al `v1.disable_v2_behavior()` in the tests main function should give the same results as running under 1.14.

1. **Enable V2 Behavior**: Now that your tests work using the v2 API, you can start looking into turning on v2 behavior. Depending on how your code is written this may require some changes. See the [Migration guide](migrate.ipynb) for details.

## Using the upgrade script


### Setup

Before getting started ensure that TensorlFlow 2.0 is installed.

In [ ]:
import tensorflow as tf

print(tf.__version__)

Clone the [tensorflow/models](https://github.com/tensorflow/models) git repository so you have some code to test on:

In [ ]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

### Read the help

The script should be installed with TensorFlow. Here is the builtin help:

In [ ]:
!tf_upgrade_v2 -h

### Example TF1 code

Here is a simple TensorFlow 1.0 script:

In [ ]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

With TensorFlow 2.0 installed it does not run:

In [ ]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### Single file

The upgrade script can be run on a single Python file:

In [ ]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

The script will print errors if it can not find a fix for the code. 

### Directory tree

Typical projects, including this simple example, will use much more than one file. Typically want to upgrade an entire package, so the script can also be run on a directory tree:

In [ ]:
# upgrade the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Note the one warning about the `dataset.make_one_shot_iterator` function.

Now the script works in with TensorFlow 2.0:

Note that because the `tf.compat.v1` module, the converted script will also run in TensorFlow 1.14. 

In [ ]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Detailed report

The script also reports a list of detailed changes. In this example it found one possibly unsafe transformation and included a warning at the top of the file: 

In [ ]:
!head -n 20 tree_report.txt

Note again the one warning about the `Dataset.make_one_shot_iterator function`.

In other cases the output will explain the reasoning for non-trivial changes:

In [ ]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [ ]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [ ]:
!cat dropout_report.txt

Here is the modified file contents, note how the script adds argument names to deal with moved and renamed arguments:

In [ ]:
!cat dropout_v2.py

A larger project might contain a few errors. For example convert the deeplab model:

In [ ]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

It produced the output files:

In [ ]:
!ls deeplab_v2

But there were errors. The report will help you pin-point what you need to fix before this will run. Here are the first three errors:

In [ ]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## "Safety" mode

The conversion script also has a less invasive `SAFETY` mode that simply changes the imports to use the `tensorflow.compat.v1` module:

In [ ]:
!cat dropout.py

In [ ]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [ ]:
!cat dropout_v2_safe.py

As you can see this doesn't upgrade your code, but does allow TensorFlow 1 code to run in TensorFlow 2

## Caveats

- Do not update parts of your code manually before running this script. In particular, functions that have had reordered arguments like `tf.argmax` or `tf.batch_to_space` cause the script to incorrectly add keyword arguments that mismap your existing code.

- The script assumes that `tensorflow` is imported using `import tensorflow as tf`.

- This script does not reorder arguments. Instead, the script adds keyword arguments to functions that have their arguments reordered.

- Check out [tf2up.ml](http://tf2up.ml) for a convenient tool to upgrade Jupyter
  notebooks and Python files in a GitHub repository.

To report upgrade script bugs or make feature requests, please file an issue on [GitHub](https://github.com/tensorflow/tensorflow/issues). And if you’re testing TensorFlow 2.0, we want to hear about it! Join the [TF 2.0 Testing community](https://groups.google.com/a/tensorflow.org/forum/#!forum/testing) and send questions and discussion to [testing@tensorflow.org](mailto:testing@tensorflow.org).